In [7]:
import pandas as pd

pd.read_parquet('./kream/231216-210342-product_card_list.parquet.gzip')

,kream_id,kream_product_img_url,kream_product_name,brand_name,trading_volume,wish,review,updated_at
0,83946,https://kream-phinf.pstatic.net/MjAyMzAyMDlfNj...,arc'teryx heliad 15 backpack black,arc'teryx,8124,17000,538,2023-12-16 21:03:42
1,44344,https://kream-phinf.pstatic.net/MjAyMTExMDJfMj...,arc'teryx squamish hoody black,arc'teryx,5260,15000,314,2023-12-16 21:03:42
2,42672,https://kream-phinf.pstatic.net/MjAyMTEwMDZfMT...,arc'teryx atom lt hoody black,arc'teryx,5499,11000,315,2023-12-16 21:03:42
3,82879,https://kream-phinf.pstatic.net/MjAyMjExMDJfMT...,arc'teryx bird head toque orca,arc'teryx,2293,10000,112,2023-12-16 21:03:42
4,84279,https://kream-phinf.pstatic.net/MjAyMzAyMDlfNT...,arc'teryx heliad 10 backpack black,arc'teryx,3494,9044,118,2023-12-16 21:03:42
...,...,...,...,...,...,...,...,...
92,101214,https://kream-phinf.pstatic.net/MjAyMzAyMTdfMj...,arc'teryx heliad 15 backpack synth,arc'teryx,100,587,4,2023-12-16 21:03:42
93,158984,https://kream-phinf.pstatic.net/MjAyMzA4MDdfMT...,arc'teryx squamish hoody pytheas,arc'teryx,94,568,8,2023-12-16 21:03:42
94,100357,https://kream-phinf.pstatic.net/MjAyMzAyMTRfNT...,arc'teryx heliad 15 backpack retreat,arc'teryx,58,563,12,2023-12-16 21:03:42
95,50101,https://kream-phinf.pstatic.net/MjAyMjAxMjVfMj...,arc'teryx beta jacket black sapphire,arc'teryx,175,559,23,2023-12-16 21:03:42


In [5]:
import sys
sys.path.append('../../../../../')

In [41]:
from typing import List, Optional
from datetime import datetime
from bs4 import BeautifulSoup
from bs4.element import Tag
from model.kream_scraping import KreamScrapingBrandSchema
def _extract_info(card: Tag):
    kream_id = card.find("a", class_="item_inner")["href"].split("/")[-1]  # type: ignore
    kream_product_img_url = card.find("img", class_="image full_width")["src"]  # type: ignore
    kream_product_name = card.find("p", class_="name").get_text(strip=True).lower()  # type: ignore
    brand_name = card.find("p", class_="product_info_brand").get_text(strip=True).lower()  # type: ignore

    updated_at = datetime.now().replace(microsecond=0)

    trading_volume = card.find("div", class_="status_value")
    if trading_volume is None:
        trading_volume = 0
    else:
        trading_volume = _convert_str_to_int(
            trading_volume.get_text(strip=True)
        )

    wish = card.find("span", class_="wish_figure")
    if wish is None:
        wish = 0
    else:
        wish = _convert_str_to_int(wish.get_text(strip=True))

    review = card.find("span", class_="review_figure")
    if review is None:
        review = 0
    else:
        review = _convert_str_to_int(review.get_text(strip=True))

    return KreamScrapingBrandSchema(
        kream_id=kream_id,  # type: ignore
        kream_product_img_url=kream_product_img_url,  # type: ignore
        kream_product_name=kream_product_name,
        brand_name=brand_name,
        trading_volume=trading_volume,
        wish=wish,
        review=review,
        updated_at=updated_at,
        
    )
def _convert_str_to_int(value: str) -> int:
        """문자열을 숫자로 변환"""

        if "거래" in value:
            value = value.replace("거래", "").replace(" ", "")

        if "만" in value:
            value = value.replace("만", "")
            return int(float(value) * 10000)

        value = value.replace(",", "")
        if value == "":
            return 0
        return int(value)

In [42]:
import json 

with open('././_temp/scrap_raw_file.json', 'r') as f:
    data = json.load(f)

In [43]:
no_data = []
for i in data :
    try :
        i = BeautifulSoup(i, 'html.parser')
        _extract_info(i)
    except Exception as e:
        print(e)
        no_data.append(i)

In [44]:
no_data

[]